In [1]:
# Packages Installation
import pandas as pd
from transformers import Trainer, TrainingArguments, T5ForConditionalGeneration, T5Tokenizer
from datasets import load_metric
from sklearn.model_selection import train_test_split
import re

C:\Users\Steffi Grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
Metadata = pd.read_csv('C:\\24592774_Dataset_Discovery_Using_LLM\\MetaData_Creation\\MetaData_Notebooks\\Prepared_MetaData_DataSet.csv')

In [8]:
Metadata.head()


,title,description,summary,tags,dataset_url,available_formats
0,"['2023w7', 'global', 'electric', 'vehicle', 'm...","['global', 'electric', 'vehicle', 'market', 's...","['editorsimple', 'original', 'visualization', ...","['makeover monday', 'cars', 'vehicles', 'elect...",https://data.world/makeovermonday/2023w7,['xlsx']
1,"['connected', 'electric', 'autonomous', 'vehic...","['connected', 'electric', 'autonomous', 'vehic...","['rolling', 'snapshot', 'data', 'collected', '...",['autonomous'],https://data.world/smartcolumbusos/650b7e59-af...,['csv']
2,"['electric', 'vehicle', 'charging', 'stations']","['dataset', 'contains', 'session', 'details', ...","['dataset', 'contains', 'session', 'details', ...","['electric vehicle', 'environment', 'energy']",https://data.world/townofcary/electric-vehicle...,"['dbf', 'csv', 'shx', 'shp', 'json', 'prj']"
3,"['nyserda', 'electric', 'vehicle', 'drive', 'c...","['new', 'york', 'state', '’', 'charge', 'ny', ...","['original', 'title', 'nyserda', 'electric', '...","['ev', 'electric vehicle', 'bev', 'phev', 'ghg...",https://data.world/data-ny-gov/thd2-fu8y,['csv']
4,"['impact', 'uncoordinated', 'plugin', 'electri...","['impact', 'uncoordinated', 'plugin', 'electri...","['original', 'title', 'impact', 'uncoordinated...","['battery', 'consumption', 'data', 'energy', '...",https://data.world/us-doe-gov/8ae7e117-313b-40...,['xlsx']


In [9]:
import pandas as pd

# Assuming your dataset is loaded into a DataFrame called 'metadata_df'
# Replace 'metadata_df' with the actual name of your DataFrame

# Check for null values in each column
null_values = Metadata.isnull().sum()
print("Null Values:")
print(null_values)

# Check for NA values in each column
na_values = Metadata.isna().sum()
print("\nNA Values:")
print(na_values)

# Check for NaN values in each column
nan_values = Metadata.isna().sum()
print("\nNaN Values:")
print(nan_values)


Null Values:
title                0
description          0
summary              0
tags                 0
dataset_url          0
available_formats    0
dtype: int64

NA Values:
title                0
description          0
summary              0
tags                 0
dataset_url          0
available_formats    0
dtype: int64

NaN Values:
title                0
description          0
summary              0
tags                 0
dataset_url          0
available_formats    0
dtype: int64


In [10]:
import torch
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertModel, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from sklearn.metrics import precision_recall_fscore_support
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Assuming MetaData_DS['description'] contains the dataset descriptions
MetaData_DS = Metadata.copy()

# Initialize DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define maximum sequence length
max_length = 128  # You can adjust this according to your dataset and memory constraints

# Tokenize dataset descriptions
tokenized_descriptions = tokenizer(list(MetaData_DS['description']), truncation=True, padding=True, max_length=max_length)

# Convert labels to numerical format
labels = [1 if url else 0 for url in MetaData_DS['dataset_url']]

# Extract input_ids and attention_mask
input_ids = tokenized_descriptions['input_ids']
attention_mask = tokenized_descriptions['attention_mask']

# Split data into training and validation sets
train_ids, val_ids, train_masks, val_masks, train_labels, val_labels = train_test_split(input_ids, attention_mask, labels, test_size=0.2, random_state=42)

# Convert tokenized texts and labels to PyTorch tensors
train_ids = torch.tensor(train_ids)
val_ids = torch.tensor(val_ids)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create TensorDataset for training and validation sets
train_dataset = TensorDataset(train_ids, train_masks, train_labels)
val_dataset = TensorDataset(val_ids, val_masks, val_labels)

# Define batch size
batch_size = 16

# Create DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the DistilBERT-based model with increased dropout
class DistilBERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(DistilBERTClassifier, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(0.3)  # Dropout rate
        self.linear = nn.Linear(self.distilbert.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state[:, 0]  # Take [CLS] token representation
        hidden_state = self.dropout(hidden_state)
        logits = self.linear(hidden_state)
        return logits

# Instantiate the DistilBERT model
num_classes = 2  # Adjust according to your task
model = DistilBERTClassifier(num_classes)

# Define the optimizer with weight decay and gradient clipping
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)
criterion = nn.CrossEntropyLoss()

# Training loop with early stopping
epochs = 10
patience = 2
best_loss = float('inf')
trigger_times = 0

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        inputs, attention_masks, labels = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs, attention_masks)
        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update model parameters
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

        # Calculate accuracy for this batch
        _, predicted = torch.max(outputs, dim=1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_train_loss = total_loss / len(train_loader)
    train_accuracy = total_correct / total_samples

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    total_correct = 0
    total_samples = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            inputs, attention_masks, labels = batch

            # Forward pass
            outputs = model(inputs, attention_masks)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            # Compute predictions
            _, predicted = torch.max(outputs, dim=1)

            # Update total samples and total correct predictions
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

            # Save predictions and labels for metric calculation
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    accuracy = total_correct / total_samples
    scheduler.step(avg_val_loss)
    
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Train Loss: {avg_train_loss}, Train Accuracy: {train_accuracy}")
    print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}")
    print(f"Validation Precision: {precision}, Validation Recall: {recall}, Validation F1 Score: {f1}")

    # Early stopping
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print('Early stopping!')
            break


C:\Users\Steffi Grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10
Train Loss: 0.5495863080024719, Train Accuracy: 0.7761194029850746
Validation Loss: 0.2011123076081276, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 2/10
Train Loss: 0.12765191234648227, Train Accuracy: 0.9850746268656716
Validation Loss: 0.013903766870498657, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 3/10
Train Loss: 0.01584861446171999, Train Accuracy: 1.0
Validation Loss: 0.004434770671650767, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 4/10
Train Loss: 0.003796592028811574, Train Accuracy: 1.0
Validation Loss: 0.001641700859181583, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 5/10
Train Loss: 0.0016678876825608313, Train Accuracy: 1.0
Validation Loss: 0.0004893661680398509, Validation Accuracy: 1.0
Validation Precision: 1.0

In [12]:
import torch
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertModel, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from sklearn.metrics import precision_recall_fscore_support
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Assuming MetaData_DS['description'] contains the dataset descriptions
MetaData_DS = Metadata.copy()

# Initialize DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define maximum sequence length
max_length = 128  # You can adjust this according to your dataset and memory constraints

# Tokenize dataset descriptions
tokenized_descriptions = tokenizer(list(MetaData_DS['description']), truncation=True, padding=True, max_length=max_length)

# Convert labels to numerical format
labels = [1 if url else 0 for url in MetaData_DS['dataset_url']]

# Extract input_ids and attention_mask
input_ids = tokenized_descriptions['input_ids']
attention_mask = tokenized_descriptions['attention_mask']

# Split data into training and validation sets
train_ids, val_ids, train_masks, val_masks, train_labels, val_labels = train_test_split(input_ids, attention_mask, labels, test_size=0.2, random_state=42)

# Convert tokenized texts and labels to PyTorch tensors
train_ids = torch.tensor(train_ids)
val_ids = torch.tensor(val_ids)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create TensorDataset for training and validation sets
train_dataset = TensorDataset(train_ids, train_masks, train_labels)
val_dataset = TensorDataset(val_ids, val_masks, val_labels)

# Define batch size
batch_size = 16

# Create DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the DistilBERT-based model with increased dropout and weight decay
class DistilBERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(DistilBERTClassifier, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(0.5)  # Increased dropout rate
        self.linear = nn.Linear(self.distilbert.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state[:, 0]  # Take [CLS] token representation
        hidden_state = self.dropout(hidden_state)
        logits = self.linear(hidden_state)
        return logits

# Instantiate the DistilBERT model
num_classes = 2  # Adjust according to your task
model = DistilBERTClassifier(num_classes)

# Define the optimizer with weight decay and gradient clipping
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)  # Increased weight decay
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)
criterion = nn.CrossEntropyLoss()

# Training loop with early stopping and gradient clipping
epochs = 10
patience = 2
best_loss = float('inf')
trigger_times = 0

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        inputs, attention_masks, labels = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs, attention_masks)
        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update model parameters
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

        # Calculate accuracy for this batch
        _, predicted = torch.max(outputs, dim=1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_train_loss = total_loss / len(train_loader)
    train_accuracy = total_correct / total_samples

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    total_correct = 0
    total_samples = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            inputs, attention_masks, labels = batch

            # Forward pass
            outputs = model(inputs, attention_masks)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            # Compute predictions
            _, predicted = torch.max(outputs, dim=1)

            # Update total samples and total correct predictions
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

            # Save predictions and labels for metric calculation
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    accuracy = total_correct / total_samples
    scheduler.step(avg_val_loss)
    
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Train Loss: {avg_train_loss}, Train Accuracy: {train_accuracy}")
    print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}")
    print(f"Validation Precision: {precision}, Validation Recall: {recall}, Validation F1 Score: {f1}")

    # Early stopping
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print('Early stopping!')
            break


C:\Users\Steffi Grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10
Train Loss: 0.5273704051971435, Train Accuracy: 0.6865671641791045
Validation Loss: 0.11749618873000145, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 2/10
Train Loss: 0.06711598448455333, Train Accuracy: 1.0
Validation Loss: 0.005186932859942317, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 3/10
Train Loss: 0.005143722286447883, Train Accuracy: 1.0
Validation Loss: 0.0016466716770082712, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 4/10
Train Loss: 0.0019688812317326665, Train Accuracy: 1.0
Validation Loss: 0.000774408457800746, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 5/10
Train Loss: 0.0008373793913051486, Train Accuracy: 1.0
Validation Loss: 0.00037346838507801294, Validation Accuracy: 1.0
Validation Precision: 1.0, Validati

In [21]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# Assuming MetaData_DS['description'] contains the dataset descriptions
MetaData_DS = Metadata.copy()

# Tokenize dataset descriptions
tokenized_descriptions = tokenizer(list(MetaData_DS['description']), truncation=True, padding=True, max_length=max_length)

# Convert tokenized texts to PyTorch tensors
dataset_ids = torch.tensor(tokenized_descriptions['input_ids'])
dataset_masks = torch.tensor(tokenized_descriptions['attention_mask'])

# Create DataLoader for dataset descriptions
dataset_loader = DataLoader(TensorDataset(dataset_ids, dataset_masks), batch_size=batch_size, shuffle=False)

# Get DistilBERT embeddings for dataset descriptions
dataset_descriptions = []
with torch.no_grad():
    for batch in dataset_loader:
        inputs, attention_masks = batch
        outputs = model(inputs, attention_masks)
        embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
        dataset_descriptions.append(embeddings.numpy())

# Convert to numpy array
dataset_descriptions = np.concatenate(dataset_descriptions, axis=0)

# Function to compute DistilBERT embeddings
def get_distilbert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=128, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
    return embeddings

# Function to predict dataset URLs based on search text
def predict_dataset_url(search_text, dataset_descriptions):
    search_emb = get_distilbert_embedding(search_text)
    
    # Reshape the search_emb array to 2D
    search_emb = search_emb.reshape(1, -1)
    
    similarities = [cosine_similarity(search_emb, dataset_emb.reshape(1, -1)) for dataset_emb in dataset_descriptions]
    
    # Find the maximum and minimum similarity values
    max_similarity = max(similarities)
    min_similarity = min(similarities)
    
    # Calculate the 90% range
    range_90 = 0.80 * (max_similarity - min_similarity)
    
    # Find the threshold value
    threshold = min_similarity + range_90
    
    # Find the indices of datasets with similarity above the threshold
    similar_indices = [i for i, sim in enumerate(similarities) if sim > threshold]
    
    results = []
    for idx in similar_indices:
        dataset_info = {
            "title": MetaData_DS.iloc[idx]['title'],
            "dataset_url": MetaData_DS.iloc[idx]['dataset_url'],
            "cosine_similarity": similarities[idx]
        }
        results.append(dataset_info)
    
    return results

# Example usage
search_text = "electric vehicles"
results = predict_dataset_url(search_text, dataset_descriptions)

# Display results
if not results:
    print("No datasets found above the 90% threshold.")
else:
    for result in results:
        print(f"Title: {result['title']}")
        print(f"Dataset URL: {result['dataset_url']}")
        print(f"Cosine Similarity: {result['cosine_similarity']}\n")



Title: ['connected', 'electric', 'autonomous', 'vehicle']
Dataset URL: https://data.world/smartcolumbusos/650b7e59-afd3-4c40-9bfc-8d614610b77b
Cosine Similarity: [[0.3919043]]

Title: ['energy', 'star', 'certified', 'electric', 'vehicle', 'supply', 'equipment']
Dataset URL: https://data.world/us-epa-gov/f3822503-5d5e-4cac-ab1c-c8c18847bc7e
Cosine Similarity: [[0.3810537]]

Title: ['electric', 'vehicles', 'charging', 'points']
Dataset URL: https://data.world/datagov-uk/e865fb83-66fb-4b5e-a95e-cfbb68a7beb5
Cosine Similarity: [[0.41192517]]



In [31]:
MetaData_DS=Metadata.copy()
MetaData_DS.head()


,title,description,summary,tags,dataset_url,available_formats
0,"['2023w7', 'global', 'electric', 'vehicle', 'm...","['global', 'electric', 'vehicle', 'market', 's...","['editorsimple', 'original', 'visualization', ...","['makeover monday', 'cars', 'vehicles', 'elect...",https://data.world/makeovermonday/2023w7,['xlsx']
1,"['connected', 'electric', 'autonomous', 'vehic...","['connected', 'electric', 'autonomous', 'vehic...","['rolling', 'snapshot', 'data', 'collected', '...",['autonomous'],https://data.world/smartcolumbusos/650b7e59-af...,['csv']
2,"['electric', 'vehicle', 'charging', 'stations']","['dataset', 'contains', 'session', 'details', ...","['dataset', 'contains', 'session', 'details', ...","['electric vehicle', 'environment', 'energy']",https://data.world/townofcary/electric-vehicle...,"['dbf', 'csv', 'shx', 'shp', 'json', 'prj']"
3,"['nyserda', 'electric', 'vehicle', 'drive', 'c...","['new', 'york', 'state', '’', 'charge', 'ny', ...","['original', 'title', 'nyserda', 'electric', '...","['ev', 'electric vehicle', 'bev', 'phev', 'ghg...",https://data.world/data-ny-gov/thd2-fu8y,['csv']
4,"['impact', 'uncoordinated', 'plugin', 'electri...","['impact', 'uncoordinated', 'plugin', 'electri...","['original', 'title', 'impact', 'uncoordinated...","['battery', 'consumption', 'data', 'energy', '...",https://data.world/us-doe-gov/8ae7e117-313b-40...,['xlsx']


In [32]:
MetaData_DS['text'] = MetaData_DS['title'] + " " + MetaData_DS['description'] + " " + MetaData_DS['summary']

In [28]:
MetaData_DS.head()

,title,description,summary,tags,dataset_url,available_formats,text
0,"['2023w7', 'global', 'electric', 'vehicle', 'm...","['global', 'electric', 'vehicle', 'market', 's...","['editorsimple', 'original', 'visualization', ...","['makeover monday', 'cars', 'vehicles', 'elect...",https://data.world/makeovermonday/2023w7,['xlsx'],"['2023w7', 'global', 'electric', 'vehicle', 'm..."
1,"['connected', 'electric', 'autonomous', 'vehic...","['connected', 'electric', 'autonomous', 'vehic...","['rolling', 'snapshot', 'data', 'collected', '...",['autonomous'],https://data.world/smartcolumbusos/650b7e59-af...,['csv'],"['connected', 'electric', 'autonomous', 'vehic..."
2,"['electric', 'vehicle', 'charging', 'stations']","['dataset', 'contains', 'session', 'details', ...","['dataset', 'contains', 'session', 'details', ...","['electric vehicle', 'environment', 'energy']",https://data.world/townofcary/electric-vehicle...,"['dbf', 'csv', 'shx', 'shp', 'json', 'prj']","['electric', 'vehicle', 'charging', 'stations'..."
3,"['nyserda', 'electric', 'vehicle', 'drive', 'c...","['new', 'york', 'state', '’', 'charge', 'ny', ...","['original', 'title', 'nyserda', 'electric', '...","['ev', 'electric vehicle', 'bev', 'phev', 'ghg...",https://data.world/data-ny-gov/thd2-fu8y,['csv'],"['nyserda', 'electric', 'vehicle', 'drive', 'c..."
4,"['impact', 'uncoordinated', 'plugin', 'electri...","['impact', 'uncoordinated', 'plugin', 'electri...","['original', 'title', 'impact', 'uncoordinated...","['battery', 'consumption', 'data', 'energy', '...",https://data.world/us-doe-gov/8ae7e117-313b-40...,['xlsx'],"['impact', 'uncoordinated', 'plugin', 'electri..."


In [35]:
import torch
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertModel, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
from sklearn.metrics import precision_recall_fscore_support
from torch.optim.lr_scheduler import ReduceLROnPlateau



# Initialize DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define maximum sequence length
max_length = 128  # You can adjust this according to your dataset and memory constraints

# Tokenize dataset descriptions
tokenized_descriptions = tokenizer(list(MetaData_DS['text']), truncation=True, padding=True, max_length=max_length)

# Convert labels to numerical format
labels = [1 if url else 0 for url in MetaData_DS['dataset_url']]

# Extract input_ids and attention_mask
input_ids = tokenized_descriptions['input_ids']
attention_mask = tokenized_descriptions['attention_mask']

# Split data into training and validation sets
train_ids, val_ids, train_masks, val_masks, train_labels, val_labels = train_test_split(input_ids, attention_mask, labels, test_size=0.2, random_state=42)

# Convert tokenized texts and labels to PyTorch tensors
train_ids = torch.tensor(train_ids)
val_ids = torch.tensor(val_ids)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create TensorDataset for training and validation sets
train_dataset = TensorDataset(train_ids, train_masks, train_labels)
val_dataset = TensorDataset(val_ids, val_masks, val_labels)

# Define batch size
batch_size = 16

# Create DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the DistilBERT-based model with increased dropout and weight decay
class DistilBERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(DistilBERTClassifier, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(0.5)  # Increased dropout rate
        self.linear = nn.Linear(self.distilbert.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state[:, 0]  # Take [CLS] token representation
        hidden_state = self.dropout(hidden_state)
        logits = self.linear(hidden_state)
        return logits

# Instantiate the DistilBERT model
num_classes = 2  # Adjust according to your task
model = DistilBERTClassifier(num_classes)

# Define the optimizer with weight decay and gradient clipping
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)  # Increased weight decay
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)
criterion = nn.CrossEntropyLoss()

# Training loop with early stopping and gradient clipping
epochs = 10
patience = 2
best_loss = float('inf')
trigger_times = 0

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        inputs, attention_masks, labels = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs, attention_masks)
        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update model parameters
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

        # Calculate accuracy for this batch
        _, predicted = torch.max(outputs, dim=1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_train_loss = total_loss / len(train_loader)
    train_accuracy = total_correct / total_samples

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    total_correct = 0
    total_samples = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            inputs, attention_masks, labels = batch

            # Forward pass
            outputs = model(inputs, attention_masks)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            # Compute predictions
            _, predicted = torch.max(outputs, dim=1)

            # Update total samples and total correct predictions
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

            # Save predictions and labels for metric calculation
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    accuracy = total_correct / total_samples
    scheduler.step(avg_val_loss)
    
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Train Loss: {avg_train_loss}, Train Accuracy: {train_accuracy}")
    print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}")
    print(f"Validation Precision: {precision}, Validation Recall: {recall}, Validation F1 Score: {f1}")

    # Early stopping
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print('Early stopping!')
            break


C:\Users\Steffi Grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10
Train Loss: 0.4420580014586449, Train Accuracy: 0.7910447761194029
Validation Loss: 0.074073800817132, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 2/10
Train Loss: 0.032543318904936315, Train Accuracy: 1.0
Validation Loss: 0.006084063556045294, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 3/10
Train Loss: 0.004791516880504787, Train Accuracy: 1.0
Validation Loss: 0.0018775270436890423, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 4/10
Train Loss: 0.0013390567270107566, Train Accuracy: 1.0
Validation Loss: 0.000665990897687152, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 5/10
Train Loss: 0.0006014390208292752, Train Accuracy: 1.0
Validation Loss: 0.0003541438636602834, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation

In [36]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')



# Tokenize dataset descriptions
tokenized_descriptions = tokenizer(list(MetaData_DS['text']), truncation=True, padding=True, max_length=max_length)

# Convert tokenized texts to PyTorch tensors
dataset_ids = torch.tensor(tokenized_descriptions['input_ids'])
dataset_masks = torch.tensor(tokenized_descriptions['attention_mask'])

# Create DataLoader for dataset descriptions
dataset_loader = DataLoader(TensorDataset(dataset_ids, dataset_masks), batch_size=batch_size, shuffle=False)

# Get DistilBERT embeddings for dataset descriptions
dataset_descriptions = []
with torch.no_grad():
    for batch in dataset_loader:
        inputs, attention_masks = batch
        outputs = model(inputs, attention_masks)
        embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
        dataset_descriptions.append(embeddings.numpy())

# Convert to numpy array
dataset_descriptions = np.concatenate(dataset_descriptions, axis=0)

# Function to compute DistilBERT embeddings
def get_distilbert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=128, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
    return embeddings

# Function to predict dataset URLs based on search text
def predict_dataset_url(search_text, dataset_descriptions):
    search_emb = get_distilbert_embedding(search_text)
    
    # Reshape the search_emb array to 2D
    search_emb = search_emb.reshape(1, -1)
    
    similarities = [cosine_similarity(search_emb, dataset_emb.reshape(1, -1)) for dataset_emb in dataset_descriptions]
    
    # Find the maximum and minimum similarity values
    max_similarity = max(similarities)
    min_similarity = min(similarities)
    
    # Calculate the 90% range
    range_90 = 0.80 * (max_similarity - min_similarity)
    
    # Find the threshold value
    threshold = min_similarity + range_90
    
    # Find the indices of datasets with similarity above the threshold
    similar_indices = [i for i, sim in enumerate(similarities) if sim > threshold]
    
    results = []
    for idx in similar_indices:
        dataset_info = {
            "title": MetaData_DS.iloc[idx]['title'],
            "dataset_url": MetaData_DS.iloc[idx]['dataset_url'],
            "cosine_similarity": similarities[idx]
        }
        results.append(dataset_info)
    
    return results

# Example usage
search_text = "electric vehicles"
results = predict_dataset_url(search_text, dataset_descriptions)

# Display results
if not results:
    print("No datasets found above the 90% threshold.")
else:
    for result in results:
        print(f"Title: {result['title']}")
        print(f"Dataset URL: {result['dataset_url']}")
        print(f"Cosine Similarity: {result['cosine_similarity']}\n")



Title: ['electric', 'vehicle', 'year']
Dataset URL: https://data.world/jeffgswanson/electric-vehicle-by-year
Cosine Similarity: [[0.3201803]]

Title: ['nys', 'electric', 'vehicles', 'data']
Dataset URL: https://data.world/john-rager/nys-electric-vehicles-data
Cosine Similarity: [[0.33808926]]

